## START

In [ ]:
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler

import re
import random
import os
import math

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
#  проверка на работу в "google colab" или "локально"

if 'sample_data' in os.listdir():
    project_dir = r'/content/drive/My Drive/Colab Notebooks/data/'
    print('Обнаружена среда выполнения Google Colab.')
    print('project_dir =>', project_dir)
else:
    project_dir = ''
    print('НЕ обнаружена среда выполнения Google Colab. Выбран режим локальной работы.')

In [ ]:
os.listdir(project_dir)

In [ ]:
y_happy = [4,20,110,15,23]
y_happy_pred = [5,15,100,9,21]

# Вычисляем MAE:
MAE = metrics.mean_absolute_error(y_happy, y_happy_pred)
print('MAE: ', MAE)

In [ ]:
#Вычисляем MSE:

MSE = metrics.mean_squared_error(y_happy, y_happy_pred)
print('MSE:', MSE)

In [ ]:
# Вычисляем коэффициент детерминации (истина VS предсказания):

R_2 = metrics.r2_score(y_happy, y_happy_pred)
print('R_2:',R_2)

In [ ]:
# Task 3.3.1

metrics.mean_squared_error([2, 3, -1, 4], [1, 3, 2, 5])


In [ ]:
#  Task 3.4.5

R_2 = metrics.r2_score([2, 3, -1, 4], [1, 3, 2, 5])
round(R_2, 2)

In [ ]:
# 3A.5 Линейная регрессия. Предобработка

data = pd.read_csv(project_dir + 'data_flats.csv', sep =';')
data.head()


In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
sns_heatmap = sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
data.price_doc.hist()

In [ ]:
data['price_doc'] = data['price_doc'].apply(lambda w: np.log(w + 1))


In [ ]:
data.price_doc.hist()

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize=(16, 16))
sns.heatmap(data.corr(), square=True, annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")


## Model


In [ ]:
df = data.copy()

In [ ]:
df.columns

In [ ]:
# X = df.drop(['id', 'price_doc', 'preschool_education_centers_raion', 'kindergarten_km', 'school_km', 'kremlin_km'], axis=1)
df.drop(['id', 'preschool_education_centers_raion', 'kindergarten_km', 'park_km', 'life_sq', 'kremlin_km'], axis=1, inplace=True)

df.dropna(inplace=True)

# 'price_doc'
Y = df['price_doc'].values
X = df.drop(['price_doc'], axis=1)

In [ ]:
X.shape
# (23925, 21)

In [ ]:
len(Y), len(X), len(X) - len(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, random_state=77)

In [ ]:
# Нормализуем обучающую и проверочную выборку по отдельности
scaler = RobustScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)


In [ ]:
%%time
# Обучаем модель на тестовом наборе данных

model = LinearRegression().fit(X_train_norm, Y_train)

In [ ]:
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
# y_pred = model.predict(X_test)

y_pred = model.predict(X_test_norm)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются

# MAE = metrics.mean_absolute_error(np.exp(Y_test) - 1, np.exp(y_pred) - 1)

MSE = metrics.mean_squared_error(np.exp(Y_test) - 1, np.exp(y_pred) - 1)

# print('MAE:', round(MAE, 0))
print('MSE:', int(round(MSE, 0)))

# MSE: 16647931507747

## END